## Resultados financeiros

1. Construir uma tabela com recomendações de compra ou não compra.
2. Construir uma tabela com recomendações de venda com acréscimo de 10 ou 30%.

Os dados para análise são encontrados em: https://www.kaggle.com/harlfoxem/housesalesprediction/version/1?select=kc_house_data.csv

In [2]:
import pandas as pd
pd.set_option('display.float_format',lambda x: '%.3f' % x)

In [3]:
#load data
data = data = pd.read_csv('datasets/kc_house_data.csv')
data.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.000,3,1.000,1180,5650,1.000,0,0,...,7,1180,0,1955,0,98178,47.511,-122.257,1340,5650
1,6414100192,20141209T000000,538000.000,3,2.250,2570,7242,2.000,0,0,...,7,2170,400,1951,1991,98125,47.721,-122.319,1690,7639
2,5631500400,20150225T000000,180000.000,2,1.000,770,10000,1.000,0,0,...,6,770,0,1933,0,98028,47.738,-122.233,2720,8062
3,2487200875,20141209T000000,604000.000,4,3.000,1960,5000,1.000,0,0,...,7,1050,910,1965,0,98136,47.521,-122.393,1360,5000
4,1954400510,20150218T000000,510000.000,3,2.000,1680,8080,1.000,0,0,...,8,1680,0,1987,0,98074,47.617,-122.045,1800,7503


### 1. Construir uma tabela com recomendações de compra ou não compra.

A partir da análise dos dados foi feita uma sugestão de compra dos imóveis. Tal sugestão está relacionada com a região, uma vez que a região interfere significativamente no preço, com a condição da casa e com a característica de ter vista para a água. 

O imóvel vai ser sugerido para compra se:
 - seu preço estiver abaixo da mediana da região;
 - se a condição da casa estiver boa e
 - se tiver vista para a água.

In [4]:
#agrupando os dados por região 

df = data[['zipcode','price']].groupby('zipcode').median().reset_index()
df2 = pd.merge(data,df,on='zipcode', how='inner')

In [5]:
#O imóvel vai ser sugerida para compra se:
# - seu preço estiver abaixo da mediana da região;
# - se a condição da casa estiver boa e
# - se tiver vista para a água.
for i in range(len(df2)):
    if (df2.loc[i,'price_x'] < df2.loc[i,'price_y']) & (df2.loc[i,'condition'] >=2) & (df2.loc[i,'waterfront'] == 1):
        df2.loc[i,'status'] = 'Compra'
    else:
        df2.loc[i,'status'] = 'Não compra'

In [6]:
df2.rename(columns = {'price_x': 'price', 'price_y': 'price_median'}, inplace = True)

In [7]:
arq_sugestao_compra = df2[['id','zipcode','price','price_median','condition','waterfront','status']]
arq_sugestao_compra

,id,zipcode,price,price_median,condition,waterfront,status
0,7129300520,98178,221900.000,278277.000,3,0,Não compra
1,4060000240,98178,205425.000,278277.000,4,0,Não compra
2,4058801670,98178,445000.000,278277.000,3,0,Não compra
3,2976800796,98178,236000.000,278277.000,3,0,Não compra
4,6874200960,98178,170000.000,278277.000,3,0,Não compra
...,...,...,...,...,...,...,...
21608,2525049086,98039,2720000.000,1892500.000,4,0,Não compra
21609,2525049113,98039,1950000.000,1892500.000,4,0,Não compra
21610,3262300485,98039,2250000.000,1892500.000,3,0,Não compra
21611,6447300365,98039,2900000.000,1892500.000,3,0,Não compra


In [8]:
#salvando em um arquivo .csv
arq_sugestao_compra.to_csv("sugestao_compra_imoveis.csv")

In [9]:
#dataframe com as casas que foram sugeridas para compra
df3 = df2[df2['status'] == 'Compra'][['id','zipcode','price','price_median','condition','waterfront','status']]
df3

,id,zipcode,price,price_median,condition,waterfront,status
10674,7631800110,98166,380000.000,390000.000,3,1,Compra
11189,2123039032,98070,369900.000,463750.000,5,1,Compra
11193,3523029041,98070,290000.000,463750.000,4,1,Compra
11207,8550001515,98070,429592.000,463750.000,5,1,Compra
11224,222029026,98070,340000.000,463750.000,5,1,Compra
11237,221029019,98070,400000.000,463750.000,3,1,Compra
11285,2923039243,98070,340000.000,463750.000,3,1,Compra
11295,2781600195,98070,285000.000,463750.000,3,1,Compra
11300,5216200090,98070,385000.000,463750.000,4,1,Compra


In [10]:
#transformando em uma tabela para colocar no read.me
from tabulate import tabulate

print(tabulate(df3, tablefmt="pipe", headers="keys"))

|       |         id |   zipcode |   price |   price_median |   condition |   waterfront | status   |
|------:|-----------:|----------:|--------:|---------------:|------------:|-------------:|:---------|
| 10674 | 7631800110 |     98166 |  380000 |         390000 |           3 |            1 | Compra   |
| 11189 | 2123039032 |     98070 |  369900 |         463750 |           5 |            1 | Compra   |
| 11193 | 3523029041 |     98070 |  290000 |         463750 |           4 |            1 | Compra   |
| 11207 | 8550001515 |     98070 |  429592 |         463750 |           5 |            1 | Compra   |
| 11224 |  222029026 |     98070 |  340000 |         463750 |           5 |            1 | Compra   |
| 11237 |  221029019 |     98070 |  400000 |         463750 |           3 |            1 | Compra   |
| 11285 | 2923039243 |     98070 |  340000 |         463750 |           3 |            1 | Compra   |
| 11295 | 2781600195 |     98070 |  285000 |         463750 |           3 |       

### 2. Construir uma tabela com recomendações de venda com acréscimo de 10 ou 30%.


Além da região, outro aspecto que influencia no preço dos imóveis é a sazonalidade. Um exemplo é que no verão os imóveis mais próximas de lagos tem uma alta de preço, já que a procura por esse tipo de estabelecimento aumenta nessa época. 

Dessa forma, foi criado uma recomendação de venda dos imóveis com base na mediana dos preços em relação a região e sazonalidade. 

Condições de venda:
   - Se o preço da compra for maior que a mediana da região + sazonalidade:
           - O preço da venda será igual ao preço da compra + 10%
   - Se o preço da compra for menor que a mediana da região + sazonalidade:
           - O preço da venda será igual ao preço da compra + 30%

In [11]:
#primeiro vou transformar a coluna 'date' para o tipo datetime
data['date'] = pd.to_datetime(data['date'])

#vou criar uma nova coluna chamada 'month' contendo somente o mês da coluna 'date'
data['month'] = data['date'].dt.month

#Verão: de junho a agosto.
#Outono: de setembro a novembro.
#Inverno: de dezembro a fevereiro.
#Primavera: de março a maio.

for i in range (len(data)):
    if data.loc[i,'month'] in [6,7,8]:
        data.loc[i,'season'] = 'summer'
    elif data.loc[i,'month'] in [9,10,11]:
        data.loc[i,'season'] = 'autumn'
    elif data.loc[i,'month'] in [12,1,2]:
        data.loc[i,'season'] = 'winter'
    else:
        data.loc[i,'season'] = 'spring'

In [12]:
data.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,month,season
0,7129300520,2014-10-13,221900.000,3,1.000,1180,5650,1.000,0,0,...,0,1955,0,98178,47.511,-122.257,1340,5650,10,autumn
1,6414100192,2014-12-09,538000.000,3,2.250,2570,7242,2.000,0,0,...,400,1951,1991,98125,47.721,-122.319,1690,7639,12,winter
2,5631500400,2015-02-25,180000.000,2,1.000,770,10000,1.000,0,0,...,0,1933,0,98028,47.738,-122.233,2720,8062,2,winter
3,2487200875,2014-12-09,604000.000,4,3.000,1960,5000,1.000,0,0,...,910,1965,0,98136,47.521,-122.393,1360,5000,12,winter
4,1954400510,2015-02-18,510000.000,3,2.000,1680,8080,1.000,0,0,...,0,1987,0,98074,47.617,-122.045,1800,7503,2,winter


In [13]:
#dataframe com a mediana do preço das casas para cada estação de cada zipcode
df4 = data[['zipcode','season','price']].groupby(['zipcode','season']).median().reset_index()
df4

,zipcode,season,price
0,98001,autumn,249900.000
1,98001,spring,262000.000
2,98001,summer,262500.000
3,98001,winter,260000.000
4,98002,autumn,235000.000
...,...,...,...
275,98198,winter,267475.000
276,98199,autumn,650000.000
277,98199,spring,709000.000
278,98199,summer,730000.000


In [14]:
#salvando em um arquivo .csv
df4.to_csv("zipcode-season.csv")

In [15]:
#df com o preço de compra e preço da mediana da região
df5 = pd.merge(data[['id','zipcode','season','price']],df4, on=['zipcode','season'], how='inner')
df5.rename(columns = {'price_x': 'price', 'price_y': 'price_median'}, inplace = True)
df5

,id,zipcode,season,price,price_median
0,7129300520,98178,autumn,221900.000,290500.000
1,2976800796,98178,autumn,236000.000,290500.000
2,1180003090,98178,autumn,190000.000,290500.000
3,2171400197,98178,autumn,350000.000,290500.000
4,1180002378,98178,autumn,299000.000,290500.000
...,...,...,...,...,...
21608,3025300250,98039,spring,1620000.000,1800000.000
21609,7397300170,98039,spring,3710000.000,1800000.000
21610,3262300920,98039,spring,1200000.000,1800000.000
21611,2470200020,98039,spring,1880000.000,1800000.000


In [16]:
df5

,id,zipcode,season,price,price_median
0,7129300520,98178,autumn,221900.000,290500.000
1,2976800796,98178,autumn,236000.000,290500.000
2,1180003090,98178,autumn,190000.000,290500.000
3,2171400197,98178,autumn,350000.000,290500.000
4,1180002378,98178,autumn,299000.000,290500.000
...,...,...,...,...,...
21608,3025300250,98039,spring,1620000.000,1800000.000
21609,7397300170,98039,spring,3710000.000,1800000.000
21610,3262300920,98039,spring,1200000.000,1800000.000
21611,2470200020,98039,spring,1880000.000,1800000.000


In [17]:
#Condições de venda:
#1. Se o preço da compra for maior que a mediana da região + sazonalidade: 
#    - O preço da venda será igual ao preço da compra + 10%
#2. Se o preço da compra for menor que a mediana da região + sazonalidade: 
#    - O preço da venda será igual ao preço da compra + 30

for i in range(len(df5)):
    if (df5.loc[i,'price'] > df5.loc[i,'price_median']):
        df5.loc[i,'price_sale'] = df5.loc[i,'price'] + (df5.loc[i,'price'])*(10/100)
        df5.loc[i,'% de lucro'] = '+10%'
    else:
        df5.loc[i,'price_sale'] = df5.loc[i,'price'] + (df5.loc[i,'price'])*(30/100)
        df5.loc[i,'% de lucro'] = '+30%'
    #lucro
    df5.loc[i,'lucro'] = df5.loc[i,'price_sale'] - df5.loc[i,'price']

In [18]:
df5

,id,zipcode,season,price,price_median,price_sale,% de lucro,lucro
0,7129300520,98178,autumn,221900.000,290500.000,288470.000,+30%,66570.000
1,2976800796,98178,autumn,236000.000,290500.000,306800.000,+30%,70800.000
2,1180003090,98178,autumn,190000.000,290500.000,247000.000,+30%,57000.000
3,2171400197,98178,autumn,350000.000,290500.000,385000.000,+10%,35000.000
4,1180002378,98178,autumn,299000.000,290500.000,328900.000,+10%,29900.000
...,...,...,...,...,...,...,...,...
21608,3025300250,98039,spring,1620000.000,1800000.000,2106000.000,+30%,486000.000
21609,7397300170,98039,spring,3710000.000,1800000.000,4081000.000,+10%,371000.000
21610,3262300920,98039,spring,1200000.000,1800000.000,1560000.000,+30%,360000.000
21611,2470200020,98039,spring,1880000.000,1800000.000,2068000.000,+10%,188000.000


In [28]:
lucro = df5['lucro'].sum()
print(f'Utilizando essa metodologia no auxílio na tomada de decisão para definição do preço de compra e revenda dos imóveis do portfólio proporcionará um lucro de U${lucro:.2f}')

Utilizando essa metodologia no auxílio na tomada de decisão para definição do preço de compra e revenda dos imóveis do portfólio proporcionará um lucro de U$2016458904.40


In [31]:
#salvando em um arquivo .csv
df5.to_csv("sugestao_preço_venda.csv")

In [30]:
#transformando em uma tabela para colocar no read.me
from tabulate import tabulate

print(tabulate(df5.head(20), tablefmt="pipe", headers="keys"))

|    |         id |   zipcode | season   |   price |   price_median |   price_sale | % de lucro   |   lucro |
|---:|-----------:|----------:|:---------|--------:|---------------:|-------------:|:-------------|--------:|
|  0 | 7129300520 |     98178 | autumn   |  221900 |         290500 |       288470 | +30%         |   66570 |
|  1 | 2976800796 |     98178 | autumn   |  236000 |         290500 |       306800 | +30%         |   70800 |
|  2 | 1180003090 |     98178 | autumn   |  190000 |         290500 |       247000 | +30%         |   57000 |
|  3 | 2171400197 |     98178 | autumn   |  350000 |         290500 |       385000 | +10%         |   35000 |
|  4 | 1180002378 |     98178 | autumn   |  299000 |         290500 |       328900 | +10%         |   29900 |
|  5 | 1180500070 |     98178 | autumn   |  335000 |         290500 |       368500 | +10%         |   33500 |
|  6 |  179000350 |     98178 | autumn   |  194000 |         290500 |       252200 | +30%         |   58200 |
|  7 | 381